In [1]:
import os
import glob

from IPython.display import display, Markdown
import pandas as pd

# Group types of models (experiment type and model type) and pick best performing in terms of f1-score per unit of analysis and report them in a table


In [2]:
df_filepaths = glob.glob('./experiment_results/*.csv')

In [3]:
exps_df = pd.concat([pd.read_csv(df_filepath) for df_filepath in df_filepaths])

In [4]:
exps_df

,language,unit_of_analysis,model_type,model_subtype,model_name,train_f1_micro,train_f1_macro,train_accuracy,train_precision_micro,train_precision_macro,train_recall_micro,train_recall_macro,test_f1_micro,test_f1_macro,test_accuracy,test_precision_micro,test_precision_macro,test_recall_micro,test_recall_macro
0,fr,title,Binary Relevance kNN,Natively Multilabel,BRkNNaClassifier,0.172792,0.114014,0.059172,0.198402,0.127886,0.154455,0.103925,0.107870,0.060714,0.000000,0.289569,0.141837,0.071429,0.043738
1,fr,title,Binary Relevance kNN,Natively Multilabel,BRkNNbClassifier,0.095207,0.074319,0.000000,0.107420,0.069964,0.213861,0.215306,0.109856,0.103092,0.000000,0.250903,0.131991,0.214286,0.255714
2,fr,title,ComplementNaiveBayes,RakelD Partitioning of labels,RakelD_ComplementNB,0.999034,0.998258,0.994083,0.998114,0.996599,1.000000,1.000000,0.317729,0.283541,0.000000,0.333701,0.298421,0.349206,0.330986
3,fr,title,LinearSVM,RakelD Partitioning of labels,RakelD_LineaSVM,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.272272,0.203039,0.000000,0.350605,0.257619,0.253968,0.192767
4,fr,title,LogisticRegressionRidge,RakelD Partitioning of labels,RakelD_LogisticRegression,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.267434,0.228633,0.000000,0.349640,0.285864,0.253968,0.224391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,en,title_and_5_sentences,XGBoost,BorderlineSMOTE,XGBoost_narrow_BorderlineSMOTE,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.603388,0.392899,0.038835,0.659811,0.450338,0.567237,0.361105
57,en,title_and_5_sentences,XGBoost,No Upsampling,XGBoost_narrow,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.602439,0.387458,0.048544,0.660854,0.451528,0.574572,0.360803
58,en,title_and_5_sentences,XGBoost,Random Oversampling,XGBoost_narrow_ROS,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.610316,0.404632,0.019417,0.655220,0.470681,0.589242,0.384710
59,en,title_and_5_sentences,XGBoost,SMOTE,XGBoost_narrow_SMOTE,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.606402,0.404920,0.038835,0.651281,0.475135,0.586797,0.380137


In [5]:
exps_df.model_type = exps_df.model_type.str.strip('¶')

In [6]:
exps_df.shape

(2409, 19)

In [7]:
def get_best_models(df, grouping_criterion):
    return df.reset_index().loc[df.reset_index().groupby(grouping_criterion)['test_f1_micro'].idxmax().dropna()]\
        .set_index(grouping_criterion)

### Generate the tables to report

In [8]:
def display_performance_table(df, metric, index_cols=['model_type'], display_=True):
    report_table = df.reset_index().copy()
    report_table['result'] = report_table[f'test_{metric}'].map(lambda x: f'{x:.3f}')
    report_table['col_title'] = report_table.unit_of_analysis.str.split('_').str.join(' ') 
    report_table['col_title'] = pd.Categorical(
        report_table.col_title,
        categories=['title', 'title and first paragraph', 'title and 5 sentences', 'title and 10 sentences',
                    'title and first sentence each paragraph', 'raw text'],
        ordered=True)
    report_table = report_table[index_cols + ['col_title', 'result']]\
        .pivot_table(index=index_cols, columns=['col_title'], values=['result'], aggfunc='first', fill_value=0)\
        .droplevel(0, axis=1)

    report_table.columns.names = [None]

    # Highlight best scoring models according to their average
    mean_perf_arr = report_table.applymap(lambda x: float(str(x).split(' ')[0])).to_numpy()
    highlight_mask = mean_perf_arr == mean_perf_arr.max()
    report_table_arr = report_table.to_numpy()  # Note it passes the array by reference
    report_table_arr[highlight_mask] = '**' + report_table_arr[highlight_mask] + '**'

    if display_:
        display(Markdown(report_table.to_markdown()))
    
    return report_table

### Generate tables for all languages

In [9]:
metrics_to_report = ['f1_micro', 'recall_micro', 'precision_micro', 'accuracy'] 

In [10]:
language_dict = {'en': 'English', 'it': 'Italian', 'fr': 'French', 'po': 'Polish', 'ru': 'Russian', 'ge': 'German'}

In [11]:
def display_metrics_and_write_to_file(df, grouping_criterion, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    best_model_dfs_list = []
    report_tables_dfs_dict = {metric: [] for metric in metrics_to_report}

    for language, results_df in df.groupby('language'):
        best_models_df = get_best_models(results_df, grouping_criterion + ['unit_of_analysis'])
        best_model_dfs_list.append(best_models_df.copy())

        display(Markdown(f'# {language_dict[language]}'))
        
        for metric in metrics_to_report:
            os.makedirs(os.path.join(output_dir, metric), exist_ok=True)

            output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
            output_dir_latex = os.path.join(output_dir, metric, 'latex')
            output_dir_csv = os.path.join(output_dir, metric, 'csv')

            os.makedirs(output_dir_markdown, exist_ok=True)
            os.makedirs(output_dir_latex, exist_ok=True)
            os.makedirs(output_dir_csv, exist_ok=True)

            display(Markdown(f'## {metric}'))

            report_table = display_performance_table(df=best_models_df, index_cols=grouping_criterion, metric=metric, display_=True)

            # Export as markdown
            markdown_file = open(os.path.join(output_dir_markdown, f"{language_dict[language]}_{metric}.md"), "w")
            report_table.reset_index().to_markdown(markdown_file, index=False)
            markdown_file.close()

            # Export as latex table
            latex_file = open(os.path.join(output_dir_latex, f"{language_dict[language]}_{metric}.tex"), "w")
            report_table.reset_index().to_latex(latex_file, index=False)
            latex_file.close()

            # Export as csv
            report_table.to_csv(os.path.join(output_dir_csv, f"{language_dict[language]}_{metric}.csv"))

            # Stack all languages into single table
            report_table['language'] = language
            report_table = report_table.reset_index().set_index(['language'] + grouping_criterion)

            report_tables_dfs_dict[metric].append(report_table)

    # Store a csv with all the metrics and parameters of the best runs per language per grouping_criterion
    pd.concat(best_model_dfs_list)\
        .reset_index().set_index(['language'] + grouping_criterion).sort_index()\
        .to_csv(os.path.join(output_dir, 'best_exp_params_metrics_per_language.csv'))

    # Report or store unified table
    display(Markdown(f'# All 6 Languages'))
    for metric in metrics_to_report:
        display(Markdown(f'## {metric}'))
        multi_language_report_table_metric = pd.concat(report_tables_dfs_dict[metric])
        display(Markdown(multi_language_report_table_metric.reset_index().to_markdown(index=False)))

        output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
        output_dir_latex = os.path.join(output_dir, metric, 'latex')
        output_dir_csv = os.path.join(output_dir, metric, 'csv')

        # Export as markdown
        markdown_file = open(os.path.join(output_dir_markdown, f"all_6_languages_{metric}.md"), "w")
        multi_language_report_table_metric.reset_index().to_markdown(markdown_file, index=False)
        markdown_file.close()

        # Export as latex table
        latex_file = open(os.path.join(output_dir_latex, f"all_6_languages_{metric}.tex"), "w")
        multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
        latex_file.close()

        # Export as csv
        multi_language_report_table_metric.to_csv(os.path.join(output_dir_csv, f"all_6_languages_{metric}.csv"))

# Per model type

In [12]:
display_metrics_and_write_to_file(df=exps_df, grouping_criterion=['model_type'], output_dir='per_model_type_tables')

# English

## f1_micro

| model_type                      |   title |   title and first paragraph |   title and 5 sentences |   title and 10 sentences |   title and first sentence each paragraph | raw text   |
|:--------------------------------|--------:|----------------------------:|------------------------:|-------------------------:|------------------------------------------:|:-----------|
| Binary Relevance kNN            |   0.31  |                       0.403 |                   0.347 |                    0.273 |                                     0.312 | 0.334      |
| ComplementNaiveBayes            |   0.574 |                       0.626 |                   0.66  |                    0.682 |                                     0.681 | 0.709      |
| Dummy Classifier                |   0.427 |                       0.459 |                   0.437 |                    0.433 |                                     0.43  | 0.435      |
| KNN                             |   0.49  |                       0.54  |                   0.527 |                    0.478 |                                     0.466 | 0.543      |
| LinearSVM                       |   0.599 |                       0.615 |                   0.642 |                    0.654 |                                     0.657 | 0.686      |
| LogisticRegression              |   0.594 |                       0.618 |                   0.628 |                    0.653 |                                     0.659 | 0.694      |
| LogisticRegressionElasticNet    |   0.604 |                       0.612 |                   0.638 |                    0.64  |                                     0.669 | 0.685      |
| LogisticRegressionLasso         |   0.555 |                       0.615 |                   0.624 |                    0.624 |                                     0.655 | 0.645      |
| LogisticRegressionRidge         |   0.61  |                       0.629 |                   0.643 |                    0.669 |                                     0.659 | 0.700      |
| Multi-label ARAM                |   0.014 |                       0     |                   0     |                    0     |                                     0     | 0          |
| Multilabel k Nearest Neighbours |   0.402 |                       0.56  |                   0.54  |                    0.553 |                                     0.533 | 0.638      |
| NaiveBayes                      |   0.631 |                       0.695 |                   0.718 |                    0.712 |                                     0.713 | **0.719**  |
| RandomForest                    |   0.581 |                       0.616 |                   0.632 |                    0.654 |                                     0.663 | 0.656      |
| RidgeClassifier                 |   0.602 |                       0.64  |                   0.636 |                    0.66  |                                     0.656 | 0.689      |
| SVM                             |   0.467 |                       0.472 |                   0.489 |                    0.523 |                                     0.541 | 0.569      |
| XGBoost                         |   0.526 |                       0.6   |                   0.615 |                    0.616 |                                     0.637 | 0.648      |

/tmp/ipykernel_32329/3322418740.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  report_table.reset_index().to_latex(latex_file, index=False)


## recall_micro

...

| language   | model_type                      |   title |   title and first paragraph |   title and 5 sentences | title and 10 sentences   | title and first sentence each paragraph   | raw text   |
|:-----------|:--------------------------------|--------:|----------------------------:|------------------------:|:-------------------------|:------------------------------------------|:-----------|
| en         | Binary Relevance kNN            |   0.352 |                       0.332 |                   0.44  | 0.389                    | 0.392                                     | 0.607      |
| en         | ComplementNaiveBayes            |   0.599 |                       0.67  |                   0.705 | 0.715                    | 0.736                                     | 0.751      |
| en         | Dummy Classifier                |   0.425 |                       0.443 |                   0.428 | 0.460                    | 0.418                                     | 0.426      |
| en         | KNN                             |   0.519 |                       0.506 |                   0.552 | 0.456                    | 0.365                                     | 0.668      |
| en         | LinearSVM                       |   0.655 |                       0.67  |                   0.715 | 0.709                    | 0.720                                     | 0.731      |
| en         | LogisticRegression              |   0.647 |                       0.669 |                   0.719 | 0.736                    | 0.723                                     | 0.751      |
| en         | LogisticRegressionElasticNet    |   0.664 |                       0.684 |                   0.729 | 0.721                    | 0.731                                     | 0.770      |
| en         | LogisticRegressionLasso         |   0.674 |                       0.679 |                   0.697 | 0.685                    | 0.702                                     | 0.714      |
| en         | LogisticRegressionRidge         |   0.645 |                       0.708 |                   0.71  | 0.737                    | 0.737                                     | 0.761      |
| en         | Multi-label ARAM                |   0.008 |                       0     |                   0     | 0                        | 0                                         | 0          |
| en         | Multilabel k Nearest Neighbours |   0.562 |                       0.584 |                   0.631 | 0.583                    | 0.614                                     | 0.677      |
| en         | NaiveBayes                      |   0.626 |                       0.697 |                   0.709 | 0.701                    | 0.698                                     | 0.707      |
| en         | RandomForest                    |   0.698 |                       0.714 |                   0.736 | 0.788                    | **0.808**                                 | 0.749      |
| en         | RidgeClassifier                 |   0.645 |                       0.69  |                   0.722 | 0.720                    | 0.721                                     | 0.748      |
| en         | SVM                             |   0.631 |                       0.759 |                   0.712 | 0.644                    | 0.699                                     | 0.719      |
| en         | XGBoost                         |   0.636 |                       0.647 |                   0.664 | 0.717                    | 0.707                                     | 0.696      |
| fr         | Binary Relevance kNN            |   0.251 |                       0.202 |                   0.136 | 0.060                    | 0.047                                     | 0.132      |
| fr         | ComplementNaiveBayes            |   0.334 |                       0.488 |                   0.409 | 0.484                    | 0.414                                     | 0.498      |
| fr         | Dummy Classifier                |   0.317 |                       0.3   |                   0.281 | 0.300                    | 0.308                                     | 0.292      |
| fr         | KNN                             |   0.292 |                       0.3   |                   0.297 | 0.295                    | 0.317                                     | 0.297      |
| fr         | LinearSVM                       |   0.351 |                       0.4   |                   0.43  | 0.383                    | 0.425                                     | 0.396      |
| fr         | LogisticRegression              |   0.364 |                       0.442 |                   0.465 | 0.394                    | 0.425                                     | 0.448      |
| fr         | LogisticRegressionElasticNet    |   0.358 |                       0.569 |                   0.443 | **0.588**                | 0.462                                     | 0.482      |
| fr         | LogisticRegressionLasso         |   0.458 |                       0.583 |                   0.523 | 0.461                    | 0.439                                     | 0.468      |
| fr         | LogisticRegressionRidge         |   0.342 |                       0.463 |                   0.442 | 0.499                    | 0.420                                     | 0.485      |
| fr         | Multilabel k Nearest Neighbours |   0.392 |                       0.32  |                   0.339 | 0.358                    | 0.190                                     | 0.275      |
| fr         | NaiveBayes                      |   0.412 |                       0.4   |                   0.423 | 0.468                    | 0.450                                     | 0.519      |
| fr         | RandomForest                    |   0.446 |                       0.399 |                   0.467 | 0.504                    | 0.412                                     | 0.405      |
| fr         | RidgeClassifier                 |   0.37  |                       0.494 |                   0.382 | 0.466                    | 0.403                                     | 0.482      |
| fr         | SVM                             |   0.341 |                       0.112 |                   0.319 | 0.070                    | 0.063                                     | 0.087      |
| fr         | XGBoost                         |   0.339 |                       0.355 |                   0.417 | 0.394                    | 0.379                                     | 0.502      |
| ge         | Binary Relevance kNN            |   0.224 |                       0.271 |                   0.213 | 0.178                    | 0.232                                     | 0.103      |
| ge         | ComplementNaiveBayes            |   0.447 |                       0.508 |                   0.508 | 0.560                    | 0.432                                     | 0.534      |
| ge         | Dummy Classifier                |   0.403 |                       0.449 |                   0.444 | 0.442                    | 0.462                                     | 0.408      |
| ge         | KNN                             |   0.433 |                       0.452 |                   0.485 | 0.507                    | 0.439                                     | 0.420      |
| ge         | LinearSVM                       |   0.477 |                       0.518 |                   0.506 | 0.532                    | 0.480                                     | 0.525      |
| ge         | LogisticRegression              |   0.474 |                       0.466 |                   0.53  | 0.518                    | 0.485                                     | 0.521      |
| ge         | LogisticRegressionElasticNet    |   0.416 |                       0.507 |                   0.489 | 0.460                    | 0.467                                     | 0.507      |
| ge         | LogisticRegressionLasso         |   0.451 |                       0.553 |                   0.524 | 0.493                    | 0.523                                     | 0.587      |
| ge         | LogisticRegressionRidge         |   0.522 |                       0.528 |                   0.501 | 0.551                    | 0.519                                     | 0.484      |
| ge         | Multilabel k Nearest Neighbours |   0.487 |                       0.494 |                   0.466 | 0.527                    | 0.418                                     | 0.378      |
| ge         | NaiveBayes                      |   0.505 |                       0.539 |                   0.557 | 0.572                    | 0.498                                     | 0.531      |
| ge         | RandomForest                    |   0.478 |                       0.59  |                   0.503 | 0.476                    | 0.550                                     | 0.496      |
| ge         | RidgeClassifier                 |   0.451 |                       0.556 |                   0.492 | 0.486                    | 0.498                                     | 0.493      |
| ge         | SVM                             |   0.396 |                       0.207 |                   0.288 | 0.441                    | 0.310                                     | 0.351      |
| ge         | XGBoost                         |   0.502 |                       0.502 |                   0.553 | 0.516                    | 0.608                                     | **0.612**  |
| it         | Binary Relevance kNN            |   0.167 |                       0.182 |                   0.171 | 0.160                    | 0.070                                     | 0.093      |
| it         | ComplementNaiveBayes            |   0.47  |                       0.472 |                   0.447 | 0.552                    | 0.453                                     | 0.552      |
| it         | Dummy Classifier                |   0.35  |                       0.362 |                   0.352 | 0.413                    | 0.324                                     | 0.352      |
| it         | KNN                             |   0.387 |                       0.357 |                   0.352 | 0.352                    | 0.348                                     | 0.353      |
| it         | LinearSVM                       |   0.465 |                       0.449 |                   0.505 | 0.575                    | 0.545                                     | 0.652      |
| it         | LogisticRegression              |   0.487 |                       0.474 |                   0.501 | 0.577                    | 0.571                                     | 0.633      |
| it         | LogisticRegressionElasticNet    |   0.472 |                       0.485 |                   0.574 | 0.563                    | 0.656                                     | 0.601      |
| it         | LogisticRegressionLasso         |   0.523 |                       0.526 |                   0.494 | 0.583                    | 0.517                                     | 0.564      |
| it         | LogisticRegressionRidge         |   0.466 |                       0.496 |                   0.502 | 0.595                    | 0.584                                     | 0.585      |
| it         | Multilabel k Nearest Neighbours |   0.469 |                       0.429 |                   0.376 | 0.404                    | 0.352                                     | 0.453      |
| it         | NaiveBayes                      |   0.488 |                       0.501 |                   0.519 | 0.583                    | 0.533                                     | 0.583      |
| it         | RandomForest                    |   0.448 |                       0.541 |                   0.514 | 0.599                    | 0.581                                     | 0.629      |
| it         | RidgeClassifier                 |   0.485 |                       0.479 |                   0.487 | 0.587                    | 0.582                                     | **0.666**  |
| it         | SVM                             |   0.474 |                       0.313 |                   0.099 | 0.453                    | 0.215                                     | 0.430      |
| it         | XGBoost                         |   0.381 |                       0.527 |                   0.411 | 0.460                    | 0.512                                     | 0.575      |
| po         | Binary Relevance kNN            |   0.337 |                       0.34  |                   0.284 | 0.258                    | 0.230                                     | 0.085      |
| po         | ComplementNaiveBayes            |   0.522 |                       0.475 |                   0.553 | 0.580                    | 0.592                                     | 0.620      |
| po         | Dummy Classifier                |   0.472 |                       0.463 |                   0.506 | 0.454                    | 0.453                                     | 0.464      |
| po         | KNN                             |   0.525 |                       0.418 |                   0.587 | 0.349                    | 0.571                                     | 0.225      |
| po         | LinearSVM                       |   0.57  |                       0.526 |                   0.579 | 0.572                    | 0.675                                     | 0.650      |
| po         | LogisticRegression              |   0.58  |                       0.534 |                   0.562 | 0.608                    | 0.632                                     | 0.750      |
| po         | LogisticRegressionElasticNet    |   0.594 |                       0.542 |                   0.581 | 0.580                    | 0.597                                     | 0.686      |
| po         | LogisticRegressionLasso         |   0.593 |                       0.583 |                   0.524 | 0.585                    | 0.632                                     | 0.617      |
| po         | LogisticRegressionRidge         |   0.609 |                       0.524 |                   0.566 | 0.595                    | 0.715                                     | 0.718      |
| po         | Multilabel k Nearest Neighbours |   0.486 |                       0.467 |                   0.496 | 0.512                    | 0.527                                     | 0.496      |
| po         | NaiveBayes                      |   0.584 |                       0.57  |                   0.583 | 0.601                    | 0.575                                     | 0.580      |
| po         | RandomForest                    |   0.639 |                       0.52  |                   0.517 | 0.558                    | 0.752                                     | **0.768**  |
| po         | RidgeClassifier                 |   0.583 |                       0.515 |                   0.562 | 0.589                    | 0.713                                     | 0.736      |
| po         | SVM                             |   0.482 |                       0.43  |                   0.422 | 0.375                    | 0.376                                     | 0.417      |
| po         | XGBoost                         |   0.523 |                       0.528 |                   0.553 | 0.537                    | 0.633                                     | 0.766      |
| ru         | Binary Relevance kNN            |   0.279 |                       0.137 |                   0.068 | 0.208                    | 0.068                                     | 0.068      |
| ru         | ComplementNaiveBayes            |   0.426 |                       0.356 |                   0.42  | 0.405                    | 0.379                                     | 0.374      |
| ru         | Dummy Classifier                |   0.212 |                       0.21  |                   0.216 | 0.219                    | 0.245                                     | 0.194      |
| ru         | KNN                             |   0.307 |                       0.236 |                   0.219 | 0.218                    | 0.219                                     | 0.219      |
| ru         | LinearSVM                       |   0.46  |                       0.375 |                   0.374 | 0.435                    | 0.375                                     | 0.496      |
| ru         | LogisticRegression              |   0.452 |                       0.344 |                   0.469 | 0.430                    | 0.387                                     | 0.527      |
| ru         | LogisticRegressionElasticNet    |   0.5   |                       0.287 |                   0.383 | 0.417                    | 0.372                                     | 0.483      |
| ru         | LogisticRegressionLasso         |   0.505 |                       0.282 |                   0.291 | **0.565**                | 0.409                                     | 0.421      |
| ru         | LogisticRegressionRidge         |   0.368 |                       0.45  |                   0.494 | 0.466                    | 0.440                                     | 0.480      |
| ru         | Multilabel k Nearest Neighbours |   0.304 |                       0.272 |                   0.281 | 0.239                    | 0.260                                     | 0.305      |
| ru         | NaiveBayes                      |   0.378 |                       0.377 |                   0.393 | 0.426                    | 0.434                                     | 0.433      |
| ru         | RandomForest                    |   0.447 |                       0.282 |                   0.378 | 0.355                    | 0.488                                     | 0.508      |
| ru         | RidgeClassifier                 |   0.397 |                       0.422 |                   0.377 | 0.405                    | 0.412                                     | 0.480      |
| ru         | SVM                             |   0.211 |                       0.163 |                   0.163 | 0.021                    | 0.157                                     | 0.020      |
| ru         | XGBoost                         |   0.352 |                       0.289 |                   0.322 | 0.378                    | 0.427                                     | 0.550      |

/tmp/ipykernel_32329/3322418740.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


## accuracy

| language   | model_type                      | title     |   title and first paragraph |   title and 5 sentences | title and 10 sentences   | title and first sentence each paragraph   | raw text   |
|:-----------|:--------------------------------|:----------|----------------------------:|------------------------:|:-------------------------|:------------------------------------------|:-----------|
| en         | Binary Relevance kNN            | 0.000     |                       0.019 |                   0.01  | 0.010                    | 0.000                                     | 0.019      |
| en         | ComplementNaiveBayes            | 0.039     |                       0.068 |                   0.068 | 0.097                    | 0.087                                     | 0.087      |
| en         | Dummy Classifier                | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| en         | KNN                             | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| en         | LinearSVM                       | 0.097     |                       0.058 |                   0.049 | 0.087                    | 0.087                                     | 0.097      |
| en         | LogisticRegression              | 0.078     |                       0.029 |                   0.078 | 0.078                    | 0.087                                     | 0.087      |
| en         | LogisticRegressionElasticNet    | 0.097     |                       0.068 |                   0.078 | 0.078                    | 0.078                                     | 0.107      |
| en         | LogisticRegressionLasso         | 0.068     |                       0.068 |                   0.078 | 0.039                    | 0.068                                     | 0.058      |
| en         | LogisticRegressionRidge         | 0.087     |                       0.068 |                   0.087 | 0.097                    | 0.097                                     | 0.087      |
| en         | Multi-label ARAM                | 0.000     |                       0     |                   0     | 0                        | 0                                         | 0          |
| en         | Multilabel k Nearest Neighbours | 0.019     |                       0.049 |                   0.049 | 0.000                    | 0.029                                     | 0.068      |
| en         | NaiveBayes                      | 0.029     |                       0.029 |                   0.039 | 0.039                    | 0.019                                     | 0.029      |
| en         | RandomForest                    | 0.068     |                       0.078 |                   0.117 | **0.146**                | 0.136                                     | **0.146**  |
| en         | RidgeClassifier                 | 0.078     |                       0.039 |                   0.087 | 0.078                    | 0.087                                     | 0.078      |
| en         | SVM                             | 0.049     |                       0.058 |                   0.058 | 0.058                    | 0.078                                     | 0.097      |
| en         | XGBoost                         | 0.029     |                       0.039 |                   0.058 | 0.087                    | 0.068                                     | 0.107      |
| fr         | Binary Relevance kNN            | 0.000     |                       0.024 |                   0     | 0.000                    | 0.000                                     | 0.000      |
| fr         | ComplementNaiveBayes            | 0.000     |                       0.024 |                   0     | 0.095                    | 0.000                                     | 0.048      |
| fr         | Dummy Classifier                | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| fr         | KNN                             | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| fr         | LinearSVM                       | 0.000     |                       0.048 |                   0.048 | 0.071                    | 0.024                                     | 0.071      |
| fr         | LogisticRegression              | 0.024     |                       0.048 |                   0.048 | 0.071                    | 0.048                                     | 0.071      |
| fr         | LogisticRegressionElasticNet    | 0.000     |                       0.048 |                   0     | 0.071                    | 0.071                                     | **0.143**  |
| fr         | LogisticRegressionLasso         | 0.000     |                       0.048 |                   0.024 | 0.024                    | 0.024                                     | 0.119      |
| fr         | LogisticRegressionRidge         | 0.000     |                       0.048 |                   0.048 | 0.071                    | 0.000                                     | 0.095      |
| fr         | Multilabel k Nearest Neighbours | 0.000     |                       0.024 |                   0.071 | 0.000                    | 0.024                                     | 0.071      |
| fr         | NaiveBayes                      | 0.024     |                       0     |                   0     | 0.000                    | 0.024                                     | 0.000      |
| fr         | RandomForest                    | 0.000     |                       0     |                   0     | 0.048                    | 0.024                                     | 0.071      |
| fr         | RidgeClassifier                 | 0.000     |                       0.048 |                   0.024 | 0.071                    | 0.000                                     | 0.095      |
| fr         | SVM                             | 0.000     |                       0     |                   0     | 0.000                    | 0.048                                     | 0.000      |
| fr         | XGBoost                         | 0.000     |                       0     |                   0.024 | 0.000                    | 0.048                                     | 0.024      |
| ge         | Binary Relevance kNN            | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ge         | ComplementNaiveBayes            | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ge         | Dummy Classifier                | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ge         | KNN                             | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ge         | LinearSVM                       | 0.000     |                       0     |                   0     | 0.029                    | 0.000                                     | 0.029      |
| ge         | LogisticRegression              | 0.000     |                       0     |                   0     | 0.029                    | 0.000                                     | 0.000      |
| ge         | LogisticRegressionElasticNet    | 0.000     |                       0.029 |                   0     | 0.000                    | 0.000                                     | 0.029      |
| ge         | LogisticRegressionLasso         | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ge         | LogisticRegressionRidge         | **0.057** |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ge         | Multilabel k Nearest Neighbours | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ge         | NaiveBayes                      | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ge         | RandomForest                    | 0.000     |                       0     |                   0     | 0.029                    | 0.000                                     | 0.029      |
| ge         | RidgeClassifier                 | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.029      |
| ge         | SVM                             | 0.000     |                       0     |                   0.029 | 0.000                    | 0.029                                     | 0.000      |
| ge         | XGBoost                         | 0.000     |                       0.029 |                   0     | 0.000                    | 0.000                                     | 0.000      |
| it         | Binary Relevance kNN            | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| it         | ComplementNaiveBayes            | 0.017     |                       0.05  |                   0.067 | 0.083                    | 0.050                                     | 0.083      |
| it         | Dummy Classifier                | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| it         | KNN                             | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| it         | LinearSVM                       | 0.033     |                       0.05  |                   0.067 | 0.067                    | 0.067                                     | 0.150      |
| it         | LogisticRegression              | 0.067     |                       0.067 |                   0.067 | 0.067                    | 0.033                                     | 0.150      |
| it         | LogisticRegressionElasticNet    | 0.033     |                       0.067 |                   0.05  | 0.100                    | 0.067                                     | **0.167**  |
| it         | LogisticRegressionLasso         | 0.017     |                       0.067 |                   0.017 | 0.083                    | 0.033                                     | 0.083      |
| it         | LogisticRegressionRidge         | 0.033     |                       0.083 |                   0.1   | 0.100                    | 0.050                                     | 0.133      |
| it         | Multilabel k Nearest Neighbours | 0.000     |                       0.017 |                   0.017 | 0.017                    | 0.017                                     | 0.033      |
| it         | NaiveBayes                      | 0.000     |                       0.017 |                   0.05  | 0.100                    | 0.033                                     | 0.067      |
| it         | RandomForest                    | 0.033     |                       0.033 |                   0.083 | 0.100                    | 0.083                                     | 0.133      |
| it         | RidgeClassifier                 | 0.017     |                       0.083 |                   0.1   | 0.083                    | 0.050                                     | 0.150      |
| it         | SVM                             | 0.050     |                       0.033 |                   0.117 | 0.067                    | 0.083                                     | 0.083      |
| it         | XGBoost                         | 0.000     |                       0.017 |                   0.017 | 0.033                    | 0.017                                     | 0.117      |
| po         | Binary Relevance kNN            | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| po         | ComplementNaiveBayes            | 0.000     |                       0     |                   0.026 | **0.077**                | 0.026                                     | 0.051      |
| po         | Dummy Classifier                | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| po         | KNN                             | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| po         | LinearSVM                       | 0.000     |                       0.026 |                   0.026 | 0.026                    | 0.000                                     | 0.026      |
| po         | LogisticRegression              | 0.000     |                       0     |                   0.026 | 0.026                    | 0.000                                     | 0.026      |
| po         | LogisticRegressionElasticNet    | 0.000     |                       0.026 |                   0     | 0.026                    | 0.000                                     | 0.000      |
| po         | LogisticRegressionLasso         | 0.000     |                       0.026 |                   0     | 0.000                    | 0.000                                     | 0.000      |
| po         | LogisticRegressionRidge         | 0.000     |                       0     |                   0.026 | 0.026                    | 0.026                                     | 0.026      |
| po         | Multilabel k Nearest Neighbours | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.026      |
| po         | NaiveBayes                      | 0.000     |                       0     |                   0.026 | 0.026                    | 0.026                                     | 0.000      |
| po         | RandomForest                    | 0.000     |                       0     |                   0.026 | 0.051                    | **0.077**                                 | 0.051      |
| po         | RidgeClassifier                 | 0.000     |                       0     |                   0.026 | 0.026                    | 0.026                                     | 0.026      |
| po         | SVM                             | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| po         | XGBoost                         | 0.000     |                       0     |                   0.026 | 0.000                    | 0.026                                     | 0.051      |
| ru         | Binary Relevance kNN            | 0.000     |                       0.053 |                   0     | 0.026                    | 0.000                                     | 0.000      |
| ru         | ComplementNaiveBayes            | 0.026     |                       0.079 |                   0.079 | 0.158                    | 0.079                                     | 0.053      |
| ru         | Dummy Classifier                | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ru         | KNN                             | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ru         | LinearSVM                       | 0.026     |                       0.026 |                   0.053 | 0.105                    | 0.105                                     | 0.132      |
| ru         | LogisticRegression              | 0.026     |                       0.053 |                   0.105 | 0.105                    | 0.053                                     | 0.079      |
| ru         | LogisticRegressionElasticNet    | 0.026     |                       0.026 |                   0.026 | 0.053                    | 0.053                                     | 0.158      |
| ru         | LogisticRegressionLasso         | 0.026     |                       0.026 |                   0.026 | 0.053                    | 0.105                                     | 0.079      |
| ru         | LogisticRegressionRidge         | 0.000     |                       0.053 |                   0.105 | 0.132                    | 0.079                                     | 0.053      |
| ru         | Multilabel k Nearest Neighbours | 0.026     |                       0     |                   0.053 | 0.132                    | 0.026                                     | 0.053      |
| ru         | NaiveBayes                      | 0.000     |                       0.026 |                   0.079 | 0.079                    | 0.026                                     | 0.053      |
| ru         | RandomForest                    | 0.026     |                       0.026 |                   0.053 | 0.026                    | 0.132                                     | 0.132      |
| ru         | RidgeClassifier                 | 0.026     |                       0.053 |                   0.053 | 0.132                    | 0.053                                     | 0.053      |
| ru         | SVM                             | 0.000     |                       0     |                   0     | 0.000                    | 0.000                                     | 0.000      |
| ru         | XGBoost                         | 0.053     |                       0     |                   0     | 0.053                    | **0.184**                                 | 0.079      |

/tmp/ipykernel_32329/3322418740.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)



# Per model sub-type

In [13]:
%%capture
display_metrics_and_write_to_file(df=exps_df, grouping_criterion=['model_type','model_subtype'], output_dir='per_model_subtype_tables')

# Per model sub-type and exp name

In [14]:
%%capture
display_metrics_and_write_to_file(df=exps_df, grouping_criterion=['model_type','model_subtype', 'model_name'], output_dir='per_exp_name_tables')